In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import pickle
import random
import os

2024-06-06 12:19:18.292552: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-06 12:19:18.293459: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-06 12:19:18.334480: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-06 12:19:18.506248: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 12:19:19.249676: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
with open(os.path.join("dataset", "train.p"), mode='rb') as training_data:
    train = pickle.load(training_data)
with open(os.path.join("dataset", "valid.p"), mode='rb') as validation_data:
    valid = pickle.load(validation_data)

X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)
X_valid, y_valid = shuffle(X_valid, y_valid)

In [ ]:
# Normalize image to [0, 1]
X_train_norm = X_train / 255
X_valid_norm = X_valid / 255

In [ ]:
import optuna
from tensorflow.keras import datasets, layers, models

def objective(trial):
    num_filters = trial.suggest_categorical('num_filters', [32, 64, 128])
    # learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    epochs = trial.suggest_categorical('epochs', [2,3,4,5,6,7,8,9,10])
    kernel_size = trial.suggest_categorical('kernel_size', [1,2,3,4])
    pool_size = trial.suggest_categorical('pool_size', [1,2,3,4])
    padding = trial.suggest_categorical('padding', ['same', 'valid'])
    n = trial.suggest_categorical('n', [1, 2, 3])
    m = trial.suggest_categorical('m', [1, 2, 3])
    o = trial.suggest_categorical('o', [0, 1, 2, 3])
    strides = trial.suggest_categorical('strides', [1, 2, 3])
    dropout = trial.suggest_categorical('dropout', [0.01, 0.05, 0.1, 0.15, 0.2, 0.3])


    model = models.Sequential()
    model.add(layers.Conv2D(filters=num_filters, kernel_size=kernel_size, padding=padding,
                            activation="relu", input_shape=(32, 32, 3)))

    for _ in range(1,m):
        for _ in range(1, n):
            model.add(layers.Conv2D( filters = num_filters , kernel_size = ( kernel_size , kernel_size ), padding = padding , activation = "relu"))
        model.add(layers.MaxPool2D(pool_size=(pool_size, pool_size), strides=strides, padding='same', data_format=None, name=None))
        model.add(layers.Dropout(dropout))

    # Other layers…
    model.add(layers.Flatten())
    for _ in range(1, o):
        model.add(layers.Dense(64, activation='relu'))

    model.add(layers.Dense(43, activation='softmax'))

    # Compile the model
    model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train_norm, y_train, epochs=epochs, validation_data=(X_valid_norm, y_valid), verbose=0)

    val_accuracy = history.history["val_accuracy"][-1]
    return val_accuracy

study = optuna.create_study(direction='maximize') # or 'minimize' for loss
study.optimize(objective, n_trials=10)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")